In [34]:
from ekpy import analysis
import numpy as np
import scipy.integrate as it

In [31]:
# load the dataset
dset = analysis.load_Dataset('./test')

# print dset, parsable collection of meta data which holds pointers to the real data files
dset

,exp_params,scope_c,scope_v,trial,filename
0,NaN,NaN,NaN,0,fe_pv__0.csv
1,NaN,NaN,NaN,1,fe_pv__1.csv
2,NaN,NaN,NaN,2,fe_pv__2.csv


In [32]:
data = dset.get_data()

In [39]:
data2 = dset.query("trial == 0").get_data()

In [44]:
list(data2['exp_params'])

[nan]

In [38]:
data.query('trial == 0')

KeyError: 'query'

In [ ]:
data['time_v']

In [ ]:
def run_function(scope, wavegen, initial_delay, pulse_delay, freq, voltage, 
                 capacitor_area, thickness, permittivity, amplification, 
                 voltage_channel:str='1', current_channel:str='2'):
    """Run function for FEHysteresis expirement. 
    
    NOTE MIGHT WANT TO ADD TO BE ABLE TO ACCEPT FOR FREQ: '10KHZ' SYNTAX LIKE BEFORE.

    args:
        wavegen (pyvisa.resources.gpib.GPIBInstrument): Keysight 81150a
        scope (pyvisa.resources.gpib.GPIBInstrument): Keysight DSOX3024A
        initial_delay (str): Intial delay in seconds
        pulse_delay (str): Same as Tdelay in seconds
        freq (str): Frequency, assumed in kHz unless a suffix is used (need to add logic)
        v_end (str): Voltage in units of volts (for vertical scale of osc), gets divided by 4 to scale voltage/div. Is later divided by loop_count to get the voltage step for each subsequent loop.
        capacitor_area (str): In units of m^2 use scientific notation (maybe add logic for suffixes)
        thickness (str): In units of m use scientific notation
        permittivity (str): Dielectric constant, gets multiplied by epsilon_0
        amplification (str): How much to amplify capacitor formula by used to calculate the current chnnl resolution which I dont use rn
    returns:
        (FE): Experiment
        
        
        notes:
        basicaly just calculate capacitance of sample from basic formula"""
    #first we initialize the meta_data
    all_args = locals()
    return all_args

In [ ]:
data.iloc[0].plot(x='time_v', y='wfm_v')

In [ ]:
data.plot()

In [ ]:
def generate_q_wfm(data_dict) -> 'dict':
    time_v = data_dict['time_v']
    wfm_v = data_dict['wfm_v']
    time_c = data_dict['time_c']
    wfm_c = data_dict['wfm_c']
    wfm_q = it.cumulative_trapezoid(wfm_c, time_c, initial=0) 
    return {}